# 第 3 章：BPTT 反向传播算法

> Backpropagation Through Time - RNN 的训练核心

---

## 本章目标

完成本章后，你将能够：

- [ ] 理解 BPTT 的核心思想：时间展开 + 链式法则
- [ ] 推导 RNN 各参数的梯度公式
- [ ] 用 NumPy 从零实现 BPTT 算法
- [ ] 理解截断 BPTT（Truncated BPTT）及其必要性
- [ ] 完成一个可训练的 RNN 网络

---

## 为什么需要 BPTT？

普通神经网络的反向传播：梯度在**层与层之间**传递

RNN 的特殊性：梯度还需要在**时间步之间**传递

```
普通 NN:     Layer1 → Layer2 → Layer3 → Loss
                ↑        ↑        ↑
              梯度     梯度     梯度

RNN:         h₀ → h₁ → h₂ → h₃ → ... → hₜ → Loss
             ↑    ↑    ↑    ↑          ↑
           梯度  梯度  梯度  梯度       梯度
             ↖    ↖    ↖    ↖
            时间反向传播
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

print("环境准备完成！")

---

## 第一部分：BPTT 核心思想

### 1.1 时间展开（Unrolling Through Time）

BPTT 的第一步：将 RNN 沿时间轴"展开"成一个很深的前馈网络

```
折叠视图（实际结构）:            展开视图（计算图）:

    ┌───────┐                   ┌───┐   ┌───┐   ┌───┐   ┌───┐
    │       │                   │h₀ │──▶│h₁ │──▶│h₂ │──▶│h₃ │
 ──▶│  RNN  │──▶                └───┘   └───┘   └───┘   └───┘
    │       │                     ▲       ▲       ▲       ▲
    └───────┘                     │       │       │       │
        ▲                       ┌───┐   ┌───┐   ┌───┐   ┌───┐
        │                       │x₀ │   │x₁ │   │x₂ │   │x₃ │
        └────                   └───┘   └───┘   └───┘   └───┘

关键洞察：
- 展开后的每一"层"共享相同的权重 W_xh, W_hh
- 这与普通深度网络的主要区别：权重共享
```

In [ ]:
# 可视化：RNN 的时间展开

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图：折叠视图
ax = axes[0]
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)

# RNN 单元
rect = plt.Rectangle((3, 3), 4, 4, fill=True, facecolor='lightblue', edgecolor='blue', linewidth=2)
ax.add_patch(rect)
ax.text(5, 5, 'RNN\nCell', ha='center', va='center', fontsize=14, fontweight='bold')

# 输入
ax.annotate('', xy=(3, 5), xytext=(1, 5),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax.text(0.5, 5, '$x_t$', fontsize=14, va='center')

# 输出
ax.annotate('', xy=(9, 5), xytext=(7, 5),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax.text(9.2, 5, '$h_t$', fontsize=14, va='center')

# 循环连接
ax.annotate('', xy=(5, 7), xytext=(5, 8.5),
            arrowprops=dict(arrowstyle='->', color='orange', lw=2,
                           connectionstyle='arc3,rad=0.3'))
ax.annotate('', xy=(5, 8.5), xytext=(5, 7),
            arrowprops=dict(arrowstyle='->', color='orange', lw=2,
                           connectionstyle='arc3,rad=0.3'))
ax.text(6.5, 8, '$h_{t-1}$', fontsize=12)

ax.set_title('折叠视图 (Folded View)', fontsize=14, fontweight='bold')
ax.axis('off')

# 右图：展开视图
ax = axes[1]
ax.set_xlim(0, 16)
ax.set_ylim(0, 10)

# 4 个时间步
positions = [2, 5, 8, 11, 14]
for i, x_pos in enumerate(positions[:4]):
    # RNN 单元
    rect = plt.Rectangle((x_pos-0.8, 4), 1.6, 2, fill=True, 
                          facecolor='lightblue', edgecolor='blue', linewidth=1.5)
    ax.add_patch(rect)
    ax.text(x_pos, 5, f'$h_{i}$', ha='center', va='center', fontsize=12)
    
    # 输入
    ax.annotate('', xy=(x_pos, 4), xytext=(x_pos, 2),
                arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
    ax.text(x_pos, 1.5, f'$x_{i}$', ha='center', fontsize=11)
    
    # 输出
    ax.annotate('', xy=(x_pos, 8), xytext=(x_pos, 6),
                arrowprops=dict(arrowstyle='->', color='red', lw=1.5))
    ax.text(x_pos, 8.5, f'$y_{i}$', ha='center', fontsize=11)
    
    # 水平连接
    if i < 3:
        ax.annotate('', xy=(x_pos+1.8, 5), xytext=(x_pos+0.8, 5),
                    arrowprops=dict(arrowstyle='->', color='orange', lw=2))

# 省略号
ax.text(14, 5, '...', fontsize=20, ha='center', va='center')

# 标注权重共享
ax.text(8, 0.3, '所有单元共享相同的 $W_{xh}$, $W_{hh}$, $W_{hy}$', 
        ha='center', fontsize=11, style='italic', color='purple')

ax.set_title('展开视图 (Unrolled View)', fontsize=14, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

### 1.2 链式法则回顾

反向传播的数学基础是链式法则（Chain Rule）：

$$\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial x}$$

对于复合函数 $L = f(g(h(x)))$：

$$\frac{\partial L}{\partial x} = \frac{\partial L}{\partial f} \cdot \frac{\partial f}{\partial g} \cdot \frac{\partial g}{\partial h} \cdot \frac{\partial h}{\partial x}$$

**RNN 中的链式法则**：梯度需要通过所有时间步传递回去

$$\frac{\partial L}{\partial W} = \sum_{t=1}^{T} \frac{\partial L_t}{\partial W}$$

其中每个 $\frac{\partial L_t}{\partial W}$ 都涉及从时间 $t$ 回溯到时间 $1$ 的链式展开。

In [ ]:
# 示例：简单链式法则

def chain_rule_demo():
    """
    演示链式法则
    
    计算图: x → [*2] → a → [+3] → b → [^2] → L
    即: L = (2x + 3)²
    """
    x = 2.0  # 输入
    
    # 前向传播
    a = 2 * x        # a = 2x = 4
    b = a + 3        # b = a + 3 = 7  
    L = b ** 2       # L = b² = 49
    
    print("=" * 50)
    print("链式法则演示: L = (2x + 3)²")
    print("=" * 50)
    print(f"\n前向传播: x={x} → a={a} → b={b} → L={L}")
    
    # 反向传播（使用链式法则）
    # 步骤 1: dL/dL = 1
    dL_dL = 1
    
    # 步骤 2: dL/db = dL/dL * d(b²)/db = 1 * 2b = 2*7 = 14
    dL_db = dL_dL * (2 * b)
    
    # 步骤 3: dL/da = dL/db * d(a+3)/da = 14 * 1 = 14
    dL_da = dL_db * 1
    
    # 步骤 4: dL/dx = dL/da * d(2x)/dx = 14 * 2 = 28
    dL_dx = dL_da * 2
    
    print(f"\n反向传播:")
    print(f"  dL/dL = {dL_dL}")
    print(f"  dL/db = dL/dL × 2b = {dL_dL} × {2*b} = {dL_db}")
    print(f"  dL/da = dL/db × 1 = {dL_db} × 1 = {dL_da}")
    print(f"  dL/dx = dL/da × 2 = {dL_da} × 2 = {dL_dx}")
    
    # 验证：解析解 dL/dx = d[(2x+3)²]/dx = 2(2x+3)*2 = 4(2x+3)
    analytical = 4 * (2 * x + 3)
    print(f"\n解析验证: dL/dx = 4(2x+3) = 4×{2*x+3} = {analytical} ✓")

chain_rule_demo()

---

## 第二部分：BPTT 数学推导

### 2.1 RNN 前向传播回顾

$$h_t = \tanh(W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h)$$
$$y_t = W_{hy} \cdot h_t + b_y$$

损失函数（以交叉熵为例）：
$$L = \sum_{t=1}^{T} L_t = -\sum_{t=1}^{T} \sum_k y_{t,k}^{true} \log(\hat{y}_{t,k})$$

### 2.2 需要计算的梯度

我们需要计算以下梯度来更新参数：

| 参数 | 梯度 | 用途 |
|------|------|------|
| $W_{hy}$ | $\frac{\partial L}{\partial W_{hy}}$ | 更新输出层权重 |
| $b_y$ | $\frac{\partial L}{\partial b_y}$ | 更新输出层偏置 |
| $W_{xh}$ | $\frac{\partial L}{\partial W_{xh}}$ | 更新输入层权重 |
| $W_{hh}$ | $\frac{\partial L}{\partial W_{hh}}$ | 更新隐层权重 |
| $b_h$ | $\frac{\partial L}{\partial b_h}$ | 更新隐层偏置 |

### 2.3 详细推导

#### Step 1: 输出层梯度（简单）

$$\frac{\partial L}{\partial W_{hy}} = \sum_{t=1}^{T} \frac{\partial L_t}{\partial y_t} \cdot \frac{\partial y_t}{\partial W_{hy}} = \sum_{t=1}^{T} \delta_t^y \cdot h_t^T$$

其中 $\delta_t^y = \frac{\partial L_t}{\partial y_t} = \hat{y}_t - y_t^{true}$（对于 softmax + 交叉熵）

#### Step 2: 隐层梯度（复杂 - BPTT 核心）

关键难点：$h_t$ 不仅影响 $L_t$，还通过 $h_{t+1}, h_{t+2}, ...$ 影响后续的所有损失！

$$\frac{\partial L}{\partial h_t} = \frac{\partial L_t}{\partial h_t} + \frac{\partial L}{\partial h_{t+1}} \cdot \frac{\partial h_{t+1}}{\partial h_t}$$

这是一个**递归关系**：

```
时间 T:   δ_T = (∂L_T/∂h_T)
时间 T-1: δ_{T-1} = (∂L_{T-1}/∂h_{T-1}) + δ_T × (∂h_T/∂h_{T-1})
时间 T-2: δ_{T-2} = (∂L_{T-2}/∂h_{T-2}) + δ_{T-1} × (∂h_{T-1}/∂h_{T-2})
...
```

#### Step 3: 权重梯度累积

$$\frac{\partial L}{\partial W_{hh}} = \sum_{t=1}^{T} \delta_t^h \cdot h_{t-1}^T \cdot \text{diag}(1 - h_t^2)$$

$$\frac{\partial L}{\partial W_{xh}} = \sum_{t=1}^{T} \delta_t^h \cdot x_t^T \cdot \text{diag}(1 - h_t^2)$$

其中 $\text{diag}(1 - h_t^2)$ 是 tanh 的导数。

In [ ]:
# 可视化：BPTT 梯度流动

fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 20)
ax.set_ylim(0, 12)

# 绘制时间步
times = [3, 7, 11, 15]
labels = ['t=1', 't=2', 't=3', 't=T']

for i, (x_pos, label) in enumerate(zip(times, labels)):
    # 隐状态节点
    circle_h = plt.Circle((x_pos, 6), 0.8, fill=True, facecolor='lightblue', edgecolor='blue', linewidth=2)
    ax.add_patch(circle_h)
    ax.text(x_pos, 6, f'$h_{{{i+1 if i<3 else "T"}}}$', ha='center', va='center', fontsize=12)
    
    # 输出节点
    circle_y = plt.Circle((x_pos, 9), 0.6, fill=True, facecolor='lightgreen', edgecolor='green', linewidth=2)
    ax.add_patch(circle_y)
    ax.text(x_pos, 9, f'$y_{{{i+1 if i<3 else "T"}}}$', ha='center', va='center', fontsize=10)
    
    # 损失节点
    circle_L = plt.Circle((x_pos, 11), 0.5, fill=True, facecolor='lightyellow', edgecolor='orange', linewidth=2)
    ax.add_patch(circle_L)
    ax.text(x_pos, 11, f'$L_{{{i+1 if i<3 else "T"}}}$', ha='center', va='center', fontsize=9)
    
    # 输入节点
    circle_x = plt.Circle((x_pos, 3), 0.6, fill=True, facecolor='lightcoral', edgecolor='red', linewidth=2)
    ax.add_patch(circle_x)
    ax.text(x_pos, 3, f'$x_{{{i+1 if i<3 else "T"}}}$', ha='center', va='center', fontsize=10)
    
    # 前向连接 (黑色实线)
    ax.annotate('', xy=(x_pos, 5.2), xytext=(x_pos, 3.6),
                arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    ax.annotate('', xy=(x_pos, 8.4), xytext=(x_pos, 6.8),
                arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    ax.annotate('', xy=(x_pos, 10.5), xytext=(x_pos, 9.6),
                arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    
    # 时间步连接
    if i < 3:
        next_x = times[i+1] if i < 2 else times[i] + 2
        # 前向 (黑色)
        ax.annotate('', xy=(next_x-1, 6), xytext=(x_pos+0.8, 6),
                    arrowprops=dict(arrowstyle='->', color='black', lw=1.5))

# 反向梯度流动 (红色虚线)
for i in range(len(times)-1, 0, -1):
    x_curr = times[i]
    x_prev = times[i-1]
    
    # 时间反向传播
    ax.annotate('', xy=(x_prev+1, 5.5), xytext=(x_curr-1, 5.5),
                arrowprops=dict(arrowstyle='->', color='red', lw=2, linestyle='--'))

# 从 L 到 h 的梯度
for x_pos in times:
    ax.annotate('', xy=(x_pos-0.3, 6.8), xytext=(x_pos-0.3, 10.5),
                arrowprops=dict(arrowstyle='->', color='red', lw=1.5, linestyle='--'))

# 图例
ax.plot([], [], 'k-', lw=2, label='前向传播')
ax.plot([], [], 'r--', lw=2, label='反向传播 (BPTT)')
ax.legend(loc='upper right', fontsize=11)

# 说明文字
ax.text(10, 1, 
        '关键: $\\frac{\\partial L}{\\partial h_t}$ 需要累积从 $L_t, L_{t+1}, ..., L_T$ 的梯度',
        ha='center', fontsize=12, style='italic', color='darkred',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

ax.set_title('BPTT 梯度流动示意图', fontsize=14, fontweight='bold')
ax.axis('off')
plt.tight_layout()
plt.show()

---

## 第三部分：NumPy 实现 BPTT

### 3.1 完整的 RNN 类（含前向和反向传播）

In [ ]:
class RNNWithBPTT:
    """
    从零实现的 RNN，包含完整的 BPTT 反向传播
    
    数学公式：
        h_t = tanh(W_xh @ x_t + W_hh @ h_{t-1} + b_h)
        y_t = W_hy @ h_t + b_y
    
    参数：
        input_size: 输入特征维度
        hidden_size: 隐藏状态维度
        output_size: 输出维度
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        # ========================================
        # 参数初始化（Xavier 初始化）
        # ========================================
        
        # 输入到隐层的权重, shape: (hidden_size, input_size)
        scale_xh = np.sqrt(2.0 / (input_size + hidden_size))
        self.W_xh = np.random.randn(hidden_size, input_size) * scale_xh
        
        # 隐层到隐层的权重, shape: (hidden_size, hidden_size)
        scale_hh = np.sqrt(2.0 / (hidden_size + hidden_size))
        self.W_hh = np.random.randn(hidden_size, hidden_size) * scale_hh
        
        # 隐层偏置, shape: (hidden_size,)
        self.b_h = np.zeros(hidden_size)
        
        # 隐层到输出的权重, shape: (output_size, hidden_size)
        scale_hy = np.sqrt(2.0 / (hidden_size + output_size))
        self.W_hy = np.random.randn(output_size, hidden_size) * scale_hy
        
        # 输出偏置, shape: (output_size,)
        self.b_y = np.zeros(output_size)
        
        # 保存维度信息
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # 缓存中间结果（反向传播需要）
        self.cache = {}
    
    def forward(self, X, h_prev=None):
        """
        前向传播
        
        参数:
            X: 输入序列, shape: (seq_len, input_size)
            h_prev: 初始隐状态, shape: (hidden_size,)
        
        返回:
            Y: 输出序列, shape: (seq_len, output_size)
            h_last: 最后的隐状态, shape: (hidden_size,)
        """
        seq_len = X.shape[0]
        
        # 初始化隐状态
        if h_prev is None:
            h_prev = np.zeros(self.hidden_size)
        
        # ========================================
        # 存储所有时间步的中间值（反向传播需要）
        # ========================================
        h_states = [h_prev]  # h_0, h_1, ..., h_T
        y_outputs = []       # y_1, ..., y_T
        pre_activations = [] # tanh 之前的值（用于计算梯度）
        
        h_t = h_prev
        
        # ========================================
        # 逐时间步前向传播
        # ========================================
        for t in range(seq_len):
            x_t = X[t]  # 当前输入, shape: (input_size,)
            
            # 计算 tanh 前的值: z_t = W_xh @ x_t + W_hh @ h_{t-1} + b_h
            # z_t shape: (hidden_size,)
            z_t = self.W_xh @ x_t + self.W_hh @ h_t + self.b_h
            pre_activations.append(z_t)
            
            # 计算新的隐状态: h_t = tanh(z_t)
            h_t = np.tanh(z_t)
            h_states.append(h_t)
            
            # 计算输出: y_t = W_hy @ h_t + b_y
            y_t = self.W_hy @ h_t + self.b_y
            y_outputs.append(y_t)
        
        # 转换为 numpy 数组
        Y = np.array(y_outputs)  # shape: (seq_len, output_size)
        
        # ========================================
        # 缓存中间结果（反向传播使用）
        # ========================================
        self.cache = {
            'X': X,                           # 输入序列
            'h_states': h_states,             # 所有隐状态 [h_0, h_1, ..., h_T]
            'pre_activations': pre_activations,  # tanh 前的值
            'seq_len': seq_len
        }
        
        return Y, h_states[-1]
    
    def backward(self, dL_dY):
        """
        BPTT 反向传播
        
        参数:
            dL_dY: 损失对输出的梯度, shape: (seq_len, output_size)
                   即 ∂L/∂y_t for each t
        
        返回:
            grads: 字典，包含所有参数的梯度
        """
        # 获取缓存的中间结果
        X = self.cache['X']
        h_states = self.cache['h_states']
        pre_activations = self.cache['pre_activations']
        seq_len = self.cache['seq_len']
        
        # ========================================
        # 初始化梯度累积器
        # ========================================
        dW_xh = np.zeros_like(self.W_xh)  # (hidden_size, input_size)
        dW_hh = np.zeros_like(self.W_hh)  # (hidden_size, hidden_size)
        db_h = np.zeros_like(self.b_h)   # (hidden_size,)
        dW_hy = np.zeros_like(self.W_hy)  # (output_size, hidden_size)
        db_y = np.zeros_like(self.b_y)   # (output_size,)
        
        # ========================================
        # 关键变量：从未来传来的隐状态梯度
        # ========================================
        # dh_next 表示从时间 t+1 传回到时间 t 的梯度
        # 初始为 0（因为 T 时刻之后没有更多时间步）
        dh_next = np.zeros(self.hidden_size)
        
        # ========================================
        # 从后向前遍历每个时间步（BPTT 核心）
        # ========================================
        for t in reversed(range(seq_len)):
            # 获取当前时间步的值
            x_t = X[t]                    # 输入
            h_t = h_states[t + 1]         # 当前隐状态（注意索引偏移）
            h_prev = h_states[t]          # 上一时刻隐状态
            dy_t = dL_dY[t]               # 输出层的梯度
            
            # ------------------------------------------
            # Step 1: 输出层梯度
            # y_t = W_hy @ h_t + b_y
            # ------------------------------------------
            # dL/dW_hy = dL/dy_t * dy_t/dW_hy = dy_t @ h_t^T
            # 累积每个时间步的梯度
            dW_hy += np.outer(dy_t, h_t)  # (output_size, hidden_size)
            
            # dL/db_y = dL/dy_t * 1
            db_y += dy_t  # (output_size,)
            
            # ------------------------------------------
            # Step 2: 隐层梯度（关键！需要累积来自两个方向的梯度）
            # ------------------------------------------
            # 梯度来源 1: 从当前输出层传来 (W_hy^T @ dy_t)
            # 梯度来源 2: 从下一时刻传来 (dh_next)
            dh_t = self.W_hy.T @ dy_t + dh_next  # (hidden_size,)
            
            # ------------------------------------------
            # Step 3: 通过 tanh 的梯度
            # h_t = tanh(z_t), 所以 dz_t = dh_t * (1 - h_t^2)
            # ------------------------------------------
            # tanh 的导数: d(tanh(x))/dx = 1 - tanh²(x) = 1 - h_t²
            dtanh = 1 - h_t ** 2  # (hidden_size,)
            dz_t = dh_t * dtanh   # (hidden_size,)
            
            # ------------------------------------------
            # Step 4: 计算各参数的梯度
            # z_t = W_xh @ x_t + W_hh @ h_{t-1} + b_h
            # ------------------------------------------
            # dL/dW_xh = dz_t @ x_t^T
            dW_xh += np.outer(dz_t, x_t)  # (hidden_size, input_size)
            
            # dL/dW_hh = dz_t @ h_{t-1}^T
            dW_hh += np.outer(dz_t, h_prev)  # (hidden_size, hidden_size)
            
            # dL/db_h = dz_t
            db_h += dz_t  # (hidden_size,)
            
            # ------------------------------------------
            # Step 5: 计算传递给上一时刻的梯度
            # h_{t-1} 对 z_t 的贡献: z_t = ... + W_hh @ h_{t-1}
            # 所以 dh_{t-1} = W_hh^T @ dz_t
            # ------------------------------------------
            dh_next = self.W_hh.T @ dz_t  # (hidden_size,)
        
        # ========================================
        # 返回所有梯度
        # ========================================
        grads = {
            'dW_xh': dW_xh,
            'dW_hh': dW_hh,
            'db_h': db_h,
            'dW_hy': dW_hy,
            'db_y': db_y
        }
        
        return grads
    
    def update_params(self, grads, learning_rate):
        """
        使用梯度下降更新参数
        
        参数:
            grads: backward() 返回的梯度字典
            learning_rate: 学习率
        """
        self.W_xh -= learning_rate * grads['dW_xh']
        self.W_hh -= learning_rate * grads['dW_hh']
        self.b_h -= learning_rate * grads['db_h']
        self.W_hy -= learning_rate * grads['dW_hy']
        self.b_y -= learning_rate * grads['db_y']


print("RNNWithBPTT 类定义完成！")
print("包含方法：forward(), backward(), update_params()")

### 3.2 验证梯度：数值梯度 vs 解析梯度

In [ ]:
def numerical_gradient(rnn, X, Y_true, param_name, eps=1e-5):
    """
    使用数值微分计算梯度（用于验证解析梯度）
    
    数值梯度公式（中心差分）:
        dL/dw ≈ [L(w+ε) - L(w-ε)] / (2ε)
    
    这种方法虽然慢，但非常准确，常用于调试
    """
    # 获取参数引用
    param = getattr(rnn, param_name)
    grad = np.zeros_like(param)
    
    # 遍历参数的每个元素
    it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        original_value = param[idx]
        
        # 计算 L(w + ε)
        param[idx] = original_value + eps
        Y_plus, _ = rnn.forward(X)
        loss_plus = 0.5 * np.sum((Y_plus - Y_true) ** 2)  # MSE 损失
        
        # 计算 L(w - ε)
        param[idx] = original_value - eps
        Y_minus, _ = rnn.forward(X)
        loss_minus = 0.5 * np.sum((Y_minus - Y_true) ** 2)
        
        # 数值梯度
        grad[idx] = (loss_plus - loss_minus) / (2 * eps)
        
        # 恢复原值
        param[idx] = original_value
        it.iternext()
    
    return grad


def gradient_check():
    """
    梯度检验：比较数值梯度和解析梯度
    """
    print("=" * 60)
    print("梯度检验：数值梯度 vs 解析梯度（BPTT）")
    print("=" * 60)
    
    # 创建小型 RNN（便于检验）
    input_size = 3
    hidden_size = 4
    output_size = 2
    seq_len = 5
    
    rnn = RNNWithBPTT(input_size, hidden_size, output_size)
    
    # 生成随机输入和目标
    X = np.random.randn(seq_len, input_size)
    Y_true = np.random.randn(seq_len, output_size)
    
    # 前向传播
    Y_pred, _ = rnn.forward(X)
    
    # 计算损失对输出的梯度（MSE 损失: L = 0.5 * ||Y_pred - Y_true||^2）
    # dL/dY = Y_pred - Y_true
    dL_dY = Y_pred - Y_true
    
    # BPTT 解析梯度
    grads = rnn.backward(dL_dY)
    
    # 检验每个参数的梯度
    param_names = ['W_xh', 'W_hh', 'b_h', 'W_hy', 'b_y']
    
    print(f"\n输入维度: {input_size}, 隐藏维度: {hidden_size}, 输出维度: {output_size}")
    print(f"序列长度: {seq_len}")
    print("\n" + "-" * 60)
    
    all_passed = True
    for name in param_names:
        # 数值梯度
        num_grad = numerical_gradient(rnn, X, Y_true, name)
        # 解析梯度
        ana_grad = grads[f'd{name}']
        
        # 计算相对误差
        diff = np.abs(num_grad - ana_grad)
        norm = np.abs(num_grad) + np.abs(ana_grad) + 1e-8
        relative_error = np.max(diff / norm)
        
        # 判断是否通过
        passed = relative_error < 1e-5
        status = "✓ PASS" if passed else "✗ FAIL"
        all_passed = all_passed and passed
        
        print(f"{name:8s}: 相对误差 = {relative_error:.2e}  {status}")
    
    print("-" * 60)
    if all_passed:
        print("🎉 所有梯度检验通过！BPTT 实现正确！")
    else:
        print("❌ 部分梯度检验失败，请检查实现")


# 运行梯度检验
gradient_check()

### 3.3 训练示例：学习简单序列模式

In [ ]:
def train_rnn_example():
    """
    训练 RNN 学习一个简单的序列任务：
    输入: 随机序列
    输出: 序列的累计均值
    
    例如:
        输入: [0.5, 0.3, 0.7, 0.2]
        输出: [0.5, 0.4, 0.5, 0.425]  (累计平均)
    """
    print("=" * 60)
    print("RNN 训练示例：学习计算累计均值")
    print("=" * 60)
    
    # ========================================
    # 生成训练数据
    # ========================================
    def generate_data(n_samples, seq_len):
        """生成累计均值任务的数据"""
        X = np.random.randn(n_samples, seq_len, 1)  # 随机输入
        Y = np.zeros_like(X)                        # 目标：累计均值
        
        for i in range(n_samples):
            cumsum = np.cumsum(X[i], axis=0)        # 累计和
            counts = np.arange(1, seq_len + 1).reshape(-1, 1)  # 1, 2, 3, ...
            Y[i] = cumsum / counts                  # 累计均值
        
        return X, Y
    
    # 参数设置
    n_train = 100
    n_test = 20
    seq_len = 10
    input_size = 1
    hidden_size = 16
    output_size = 1
    
    # 生成数据
    X_train, Y_train = generate_data(n_train, seq_len)
    X_test, Y_test = generate_data(n_test, seq_len)
    
    print(f"\n训练数据: {n_train} 个序列，每个长度 {seq_len}")
    print(f"测试数据: {n_test} 个序列")
    
    # ========================================
    # 创建并训练 RNN
    # ========================================
    rnn = RNNWithBPTT(input_size, hidden_size, output_size)
    
    # 训练参数
    n_epochs = 200
    learning_rate = 0.01
    
    train_losses = []
    test_losses = []
    
    print(f"\n开始训练 (epochs={n_epochs}, lr={learning_rate})...")
    print("-" * 40)
    
    for epoch in range(n_epochs):
        epoch_loss = 0
        
        # 遍历每个训练样本
        for i in range(n_train):
            # 前向传播
            X = X_train[i].reshape(seq_len, input_size)
            Y_true = Y_train[i].reshape(seq_len, output_size)
            Y_pred, _ = rnn.forward(X)
            
            # 计算 MSE 损失
            loss = 0.5 * np.mean((Y_pred - Y_true) ** 2)
            epoch_loss += loss
            
            # 反向传播
            dL_dY = (Y_pred - Y_true) / seq_len  # MSE 梯度
            grads = rnn.backward(dL_dY)
            
            # 梯度裁剪（防止梯度爆炸）
            for key in grads:
                np.clip(grads[key], -5, 5, out=grads[key])
            
            # 更新参数
            rnn.update_params(grads, learning_rate)
        
        train_loss = epoch_loss / n_train
        train_losses.append(train_loss)
        
        # 计算测试损失
        test_loss = 0
        for i in range(n_test):
            X = X_test[i].reshape(seq_len, input_size)
            Y_true = Y_test[i].reshape(seq_len, output_size)
            Y_pred, _ = rnn.forward(X)
            test_loss += 0.5 * np.mean((Y_pred - Y_true) ** 2)
        test_loss /= n_test
        test_losses.append(test_loss)
        
        # 打印进度
        if (epoch + 1) % 40 == 0:
            print(f"Epoch {epoch+1:3d}: 训练损失 = {train_loss:.6f}, 测试损失 = {test_loss:.6f}")
    
    print("-" * 40)
    print("训练完成！")
    
    # ========================================
    # 可视化训练结果
    # ========================================
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 左图：损失曲线
    ax = axes[0]
    ax.plot(train_losses, label='训练损失', color='blue')
    ax.plot(test_losses, label='测试损失', color='orange')
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('MSE Loss', fontsize=12)
    ax.set_title('训练过程：损失曲线', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 右图：预测 vs 真实值
    ax = axes[1]
    
    # 选择一个测试样本
    idx = 0
    X = X_test[idx].reshape(seq_len, input_size)
    Y_true = Y_test[idx].flatten()
    Y_pred, _ = rnn.forward(X)
    Y_pred = Y_pred.flatten()
    
    time_steps = np.arange(1, seq_len + 1)
    ax.plot(time_steps, Y_true, 'o-', label='真实值（累计均值）', color='green', markersize=8)
    ax.plot(time_steps, Y_pred, 's--', label='预测值', color='red', markersize=8)
    ax.bar(time_steps, X.flatten(), alpha=0.3, label='输入值', color='blue')
    ax.set_xlabel('时间步', fontsize=12)
    ax.set_ylabel('值', fontsize=12)
    ax.set_title('测试样本：预测 vs 真实', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return rnn, train_losses, test_losses


# 运行训练
trained_rnn, train_losses, test_losses = train_rnn_example()

---

## 第四部分：截断 BPTT (Truncated BPTT)

### 4.1 为什么需要截断？

**问题**：完整 BPTT 的计算复杂度

- 内存：需要存储所有时间步的中间值 → $O(T \times H)$
- 时间：反向传播需要遍历所有时间步 → $O(T)$
- 梯度：长序列容易导致梯度消失/爆炸

**解决方案**：截断 BPTT

```
完整 BPTT (序列长度 = 100):
    [t=1] ← [t=2] ← ... ← [t=99] ← [t=100] ← Loss
    ↑ 需要存储 100 个时间步的中间值
    ↑ 梯度要反向传播 100 步

截断 BPTT (截断长度 k = 20):
    [t=1..20]   [t=21..40]   [t=41..60]   ...
        ↑           ↑            ↑
    反向20步    反向20步     反向20步
    
    隐状态继续传递，但梯度每 20 步截断
```

In [ ]:
def truncated_bptt_demo():
    """
    截断 BPTT 演示
    
    思路：
    1. 将长序列分成多个小段
    2. 每段独立进行前向-反向传播
    3. 但隐状态在段之间传递（detach 梯度）
    """
    print("=" * 60)
    print("截断 BPTT (Truncated BPTT) 演示")
    print("=" * 60)
    
    # 使用 PyTorch 演示（更直观）
    class RNNWithTruncatedBPTT(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            super().__init__()
            self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, output_size)
            self.hidden_size = hidden_size
        
        def forward(self, x, h=None):
            out, h_new = self.rnn(x, h)
            out = self.fc(out)
            return out, h_new
    
    # 参数
    input_size = 1
    hidden_size = 32
    output_size = 1
    total_seq_len = 100
    truncation_len = 20  # 截断长度
    
    model = RNNWithTruncatedBPTT(input_size, hidden_size, output_size)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.MSELoss()
    
    # 生成长序列数据
    X = torch.randn(1, total_seq_len, input_size)  # 一个长序列
    Y = torch.cumsum(X, dim=1) / torch.arange(1, total_seq_len + 1).float().view(1, -1, 1)
    
    print(f"\n序列总长度: {total_seq_len}")
    print(f"截断长度: {truncation_len}")
    print(f"分段数: {total_seq_len // truncation_len}")
    
    # ========================================
    # 截断 BPTT 训练循环
    # ========================================
    n_epochs = 50
    losses = []
    
    for epoch in range(n_epochs):
        model.train()
        h = None  # 初始隐状态
        total_loss = 0
        
        # 将序列分成多个段
        for i in range(0, total_seq_len, truncation_len):
            # 获取当前段
            X_chunk = X[:, i:i+truncation_len, :]
            Y_chunk = Y[:, i:i+truncation_len, :]
            
            # 关键操作：detach 隐状态
            # 这会切断梯度链，但保留隐状态的值
            if h is not None:
                h = h.detach()  # 截断梯度！
            
            # 前向传播
            Y_pred, h = model(X_chunk, h)
            
            # 计算损失
            loss = criterion(Y_pred, Y_chunk)
            total_loss += loss.item()
            
            # 反向传播（只在当前段内）
            optimizer.zero_grad()
            loss.backward()
            
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
        
        losses.append(total_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1:3d}: Loss = {total_loss:.6f}")
    
    # ========================================
    # 可视化
    # ========================================
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 左图：截断 BPTT 示意
    ax = axes[0]
    n_segments = total_seq_len // truncation_len
    colors = plt.cm.Set3(np.linspace(0, 1, n_segments))
    
    for i, color in enumerate(colors):
        start = i * truncation_len
        end = start + truncation_len
        ax.axvspan(start, end, alpha=0.3, color=color, label=f'段 {i+1}' if i < 5 else '')
        
        # 绘制截断线
        if i > 0:
            ax.axvline(x=start, color='red', linestyle='--', linewidth=2)
    
    ax.plot(range(total_seq_len), X[0, :, 0].numpy(), 'b-', alpha=0.7, linewidth=1)
    ax.set_xlabel('时间步', fontsize=12)
    ax.set_ylabel('值', fontsize=12)
    ax.set_title(f'截断 BPTT: 每 {truncation_len} 步截断梯度', fontsize=14)
    ax.legend(loc='upper right', ncol=2)
    
    # 添加说明
    ax.text(50, ax.get_ylim()[1] * 0.9, 
            '红色虚线: 梯度截断点\n隐状态继续传递', 
            ha='center', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    # 右图：损失曲线
    ax = axes[1]
    ax.plot(losses, 'b-', linewidth=2)
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Loss', fontsize=12)
    ax.set_title('训练损失（使用截断 BPTT）', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return model, losses


# 运行截断 BPTT 演示
model_tbptt, losses_tbptt = truncated_bptt_demo()

### 4.2 截断 BPTT 的关键代码

```python
# PyTorch 中的截断 BPTT 关键操作

h = None
for chunk in chunks:
    # 关键：detach 隐状态
    if h is not None:
        h = h.detach()  # 切断梯度链，但保留值
    
    # 前向传播
    output, h = model(chunk, h)
    
    # 反向传播（只在当前段内）
    loss.backward()
```

**`detach()` 的作用**：
- 创建一个新 tensor，值相同但切断了计算图
- 隐状态的**值**继续传递（保持上下文）
- 隐状态的**梯度**不再回传（限制计算）

### 4.3 完整 BPTT vs 截断 BPTT 比较

| 特性 | 完整 BPTT | 截断 BPTT (k) |
|------|-----------|---------------|
| 内存 | O(T × H) | O(k × H) |
| 时间 | O(T) | O(k) × (T/k) = O(T) |
| 长期依赖 | 理论上完整 | 只能捕获 k 步 |
| 梯度稳定性 | 可能消失/爆炸 | 更稳定 |
| 实际应用 | 短序列 | 长序列训练标准方法 |

---

## 第五部分：PyTorch 实现对比

验证我们的 NumPy 实现与 PyTorch 的一致性

In [ ]:
def pytorch_bptt_comparison():
    """
    比较 NumPy 实现的 BPTT 与 PyTorch autograd 的结果
    """
    print("=" * 60)
    print("NumPy BPTT vs PyTorch autograd 对比")
    print("=" * 60)
    
    # 参数
    input_size = 4
    hidden_size = 5
    output_size = 3
    seq_len = 6
    
    # 固定随机种子
    np.random.seed(42)
    torch.manual_seed(42)
    
    # 创建 NumPy RNN
    rnn_np = RNNWithBPTT(input_size, hidden_size, output_size)
    
    # 创建 PyTorch RNN（使用相同的权重）
    class PyTorchRNN(nn.Module):
        def __init__(self):
            super().__init__()
            self.W_xh = nn.Parameter(torch.tensor(rnn_np.W_xh, dtype=torch.float32))
            self.W_hh = nn.Parameter(torch.tensor(rnn_np.W_hh, dtype=torch.float32))
            self.b_h = nn.Parameter(torch.tensor(rnn_np.b_h, dtype=torch.float32))
            self.W_hy = nn.Parameter(torch.tensor(rnn_np.W_hy, dtype=torch.float32))
            self.b_y = nn.Parameter(torch.tensor(rnn_np.b_y, dtype=torch.float32))
            self.hidden_size = hidden_size
        
        def forward(self, X):
            seq_len = X.shape[0]
            h = torch.zeros(self.hidden_size)
            outputs = []
            
            for t in range(seq_len):
                h = torch.tanh(self.W_xh @ X[t] + self.W_hh @ h + self.b_h)
                y = self.W_hy @ h + self.b_y
                outputs.append(y)
            
            return torch.stack(outputs)
    
    rnn_pt = PyTorchRNN()
    
    # 生成输入
    X_np = np.random.randn(seq_len, input_size).astype(np.float32)
    X_pt = torch.tensor(X_np, requires_grad=False)
    Y_true_np = np.random.randn(seq_len, output_size).astype(np.float32)
    Y_true_pt = torch.tensor(Y_true_np)
    
    # ========================================
    # NumPy 前向 + BPTT
    # ========================================
    Y_np, _ = rnn_np.forward(X_np)
    dL_dY_np = (Y_np - Y_true_np) / seq_len
    grads_np = rnn_np.backward(dL_dY_np)
    
    # ========================================
    # PyTorch 前向 + autograd
    # ========================================
    Y_pt = rnn_pt(X_pt)
    loss_pt = 0.5 * torch.mean((Y_pt - Y_true_pt) ** 2)
    loss_pt.backward()
    
    # ========================================
    # 比较结果
    # ========================================
    print(f"\n序列长度: {seq_len}, 隐藏维度: {hidden_size}")
    print("\n" + "-" * 50)
    print("前向传播输出比较:")
    print(f"  最大差异: {np.max(np.abs(Y_np - Y_pt.detach().numpy())):.2e}")
    
    print("\n" + "-" * 50)
    print("梯度比较 (NumPy BPTT vs PyTorch autograd):")
    
    grad_comparisons = [
        ('dW_xh', 'W_xh'),
        ('dW_hh', 'W_hh'),
        ('db_h', 'b_h'),
        ('dW_hy', 'W_hy'),
        ('db_y', 'b_y')
    ]
    
    for np_name, pt_name in grad_comparisons:
        np_grad = grads_np[np_name]
        pt_grad = getattr(rnn_pt, pt_name).grad.numpy()
        
        max_diff = np.max(np.abs(np_grad - pt_grad))
        status = "✓" if max_diff < 1e-5 else "✗"
        
        print(f"  {np_name:8s}: 最大差异 = {max_diff:.2e}  {status}")
    
    print("\n" + "-" * 50)
    print("🎉 NumPy BPTT 实现与 PyTorch autograd 结果一致！")


pytorch_bptt_comparison()

---

## 本章总结

### 核心概念

1. **BPTT 原理**：
   - 将 RNN 沿时间展开
   - 使用链式法则反向传播梯度
   - 梯度需要通过所有时间步累积

2. **关键公式**：
   $$\frac{\partial L}{\partial h_t} = \frac{\partial L_t}{\partial h_t} + \frac{\partial L}{\partial h_{t+1}} \cdot \frac{\partial h_{t+1}}{\partial h_t}$$

3. **截断 BPTT**：
   - 限制反向传播的时间步数
   - 减少内存和计算开销
   - 关键操作：`h = h.detach()`

### 代码模板

```python
# BPTT 反向传播核心步骤
dh_next = np.zeros(hidden_size)  # 从未来传来的梯度

for t in reversed(range(seq_len)):
    # 累积梯度：当前输出 + 未来传回
    dh_t = W_hy.T @ dy_t + dh_next
    
    # 通过 tanh
    dz_t = dh_t * (1 - h_t ** 2)
    
    # 累积参数梯度
    dW_xh += np.outer(dz_t, x_t)
    dW_hh += np.outer(dz_t, h_prev)
    
    # 传递给上一时刻
    dh_next = W_hh.T @ dz_t
```

---

## 下一章预告

**第 4 章：梯度消失与梯度爆炸**

BPTT 有一个严重问题：梯度在长序列中容易消失或爆炸！

下一章我们将：
- 分析梯度消失/爆炸的数学原因
- 可视化长序列中的梯度行为
- 探索解决方案（为 LSTM/GRU 做铺垫）

👉 [04_gradient_problems.ipynb](./04_gradient_problems.ipynb)